# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving informaton out of natural language (could be seqeuences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

> 📖 **Resource:** See all course materials, resources and extra-curriculum for this notebook on GitHub: https://github.com/mrdbourke/tensorflow-deep-learning/

Video N° 225: Preparing a notebook for our first NLP with TensorFlow project

## Check for GPU

In [1]:
import numpy as np

from MachineLearningUtils.system_setup.gpu import activate_gpu
activate_gpu()

2024-04-25 07:18:30.625779: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 07:18:30.625816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 07:18:30.650590: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 07:18:30.709469: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


1 Physical GPUs, 1 Logical GPU


2024-04-25 07:18:33.338713: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 07:18:33.553577: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 07:18:33.553772: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
!nvidia-smi -L

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
GPU 0: NVIDIA GeForce RTX 3050 (UUID: GPU-ee6cd8ff-34bf-dc58-2edb-4f5120aaec55)


## Get helper functions

In [3]:
from MachineLearningUtils.data_acquisition.data_downloader import *
from MachineLearningUtils.data_visualization.augmentation_effects import *
from MachineLearningUtils.data_visualization.evaluation_metrics import *
from MachineLearningUtils.data_visualization.image_visualization import *
from MachineLearningUtils.data_visualization.model_learning_curves import *
from MachineLearningUtils.data_visualization.prediction_visualization import *
from MachineLearningUtils.training_utilities.model_callbacks import *
from MachineLearningUtils.training_utilities.transfer_learning import *

## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster).

See the original source here: https://www.kaggle.com/c/nlp-getting-started

In [4]:
url = "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
download_data(url=url, file_path="nlp_getting_started.zip", extract=True)

The file nlp_getting_started.zip already exists.
Extracting nlp_getting_started.zip as ZIP...
nlp_getting_started.zip has been extracted to current directory.


Video N°226: Becoming one with the data and visualising a text dataset

## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use Python: https://realpython.com/read-write-files-python/

But I prefer to get visual straight away.

So another way to do this is to use pandas...

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class?
train_df_shuffled.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df) - 5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target= row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 1 (real disaster)
Text:
10News ? Water main break disrupts trolley service http://t.co/pAug7a68i0

---

Target: 1 (real disaster)
Text:
#Amsterdam POV video captures violent landing at Amsterdam Airport Schiphol during a st... http://t.co/AlUMrGl40e http://t.co/8h2KCTFB8I

---

Target: 0 (not real disaster)
Text:
DISASTER AVERTED: Police kill gunman with Û÷hoax deviceÛª atåÊcinema http://t.co/5NG0FzpVdS

---

Target: 0 (not real disaster)
Text:
Yo I got bars and I'm not even a rapper

---

Target: 0 (not real disaster)
Text:
If you sit and rant on snapchat to your apparent fans when you have about 8000 followers I hope your in a train crash xoxo

---


Video N°227: Splitting data into training and validation sets

### Split data into training and validation sets

We want to be able to see how our model is performing on unseen data whilst it trains.

And because the testing dataset doesn't have labels, we'll have to create a validation dataset to evaluate on (the model won't see the validation dataset during training so we can use its samples and labels to evaluate our model's performance).

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation split
                                                                            random_state=42)

In [13]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

Video N°228: Converting text data to numbers using tokenisation and embeddings (overview)
Video N°229: Setting up a TensorFlow TextVectorization layer to convert text to numbers

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=10000, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be?
                                    pad_to_max_tokens=True)

In [17]:
train_sentences[0].split(), len(train_sentences[0].split())

(['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet'], 7)

In [18]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [19]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

Video N°230: Mapping the TextVectorization layer to text data and turning it into numbers

In [20]:
# Fit the text vectorizer instance to the training data using the adapt() method
text_vectorizer.adapt(train_sentences)

In [21]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
@GrabakaHitman @Izi_Garcia when he flattened machida...did he lose that fight..nope he lost fights to guys he shouldn't of lost to also        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,    1,   45,   56,  485,    1,   56, 1505,   16,    1,   56,
         681, 3001,    5,  576]])>

In [23]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab: {bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [24]:
# how long is our vocab?
len(words_in_vocab)

10000

Video N°231: Creating an Embedding layer to turn tokenised text into embedding vectors

## Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use TensorFlow's embedding layer:
[https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)
The parameters we care most about for our embedding layer:
- `input_dim` = the size of our vocabulary
- `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
- `input_length` = length of the sequences being passed to the embedding layer

In [25]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set the input shape
                             output_dim=128, # set the size of the embedding vector
                             embeddings_initializer="uniform", # default, initialize embedding vectors randomly
                             input_length=max_length) # how long is each input
embedding

In [26]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence}')
print(f'\n\nEmbedded version:')

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Hibernating pbx irrespective of pitch fatality careerism pan: crbZFZ


Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00073292,  0.03469774, -0.04441135, ...,  0.01416595,
         -0.00486546,  0.0440566 ],
        [-0.02556345,  0.02247613,  0.0228686 , ..., -0.01249836,
          0.00860999, -0.03668206],
        [-0.00073292,  0.03469774, -0.04441135, ...,  0.01416595,
         -0.00486546,  0.0440566 ],
        ...,
        [ 0.02461983, -0.02596474, -0.02922946, ..., -0.02134675,
          0.01060081,  0.01780658],
        [ 0.02461983, -0.02596474, -0.02922946, ..., -0.02134675,
          0.01060081,  0.01780658],
        [ 0.02461983, -0.02596474, -0.02922946, ..., -0.02134675,
          0.01060081,  0.01780658]]], dtype=float32)>

In [27]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00073292,  0.03469774, -0.04441135, -0.04653853, -0.00745245,
        -0.02196614,  0.037846  , -0.0123395 ,  0.00701945, -0.03935646,
        -0.01026077,  0.0421334 , -0.01159269,  0.02559427, -0.03343344,
        -0.04209521,  0.04020859,  0.00952066,  0.0441342 ,  0.01727689,
        -0.01675706,  0.01039745, -0.01201605,  0.04004662, -0.03002352,
        -0.0260776 , -0.00976036,  0.01136826, -0.01386015,  0.02963423,
        -0.0329824 , -0.0373079 ,  0.02577973, -0.03623586, -0.01246981,
         0.01436687,  0.03395281,  0.02409868,  0.04986032,  0.00584126,
        -0.01905798,  0.02327761, -0.00627295,  0.04557842, -0.0477283 ,
         0.01180729, -0.0462804 , -0.00633027,  0.00261569,  0.0002694 ,
         0.01335653, -0.02890294,  0.03307437, -0.03175048,  0.01979259,
         0.04305634,  0.03400102,  0.00584256,  0.01821729,  0.04533894,
        -0.02732452,  0.00714255,  0.01151583,  0.01530791, -0.00369186,
  

Video N°232: Discussing the various modelling experiments we're going to run

## Modelling a text dataset (setting up our modelling experiments)

Now we've a got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline) - got this from here: [https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Word Embedding (feature extractor) (using transfer learning for NLP)
* Model 7: Same as model 6 but using 10% of data

How are we going to approach all of these?
Use the standard steps in modelling with tensorflow:
* Create a model
* Build a model
* Fit a model
* Evaluate our model

Video N°233: Model 0:Building a basline model to try and improve upon

### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> 🔑 **Note:** It's common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [28]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
    ("clf", MultinomialNB()) # model the text converted to numbers
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

CPU times: user 263 ms, sys: 9.35 ms, total: 273 ms
Wall time: 278 ms


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [30]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [31]:
train_labels

array([0, 0, 1, ..., 1, 1, 0])

Video N°234: Creating a function to track and evaluate our model's results

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however, this will be cumbersome and could easily be fixed with a function

Let's create one to compare our model's predictions with the truth labels using the following metrics:
 
* Accuracy
* Precision
* Recall
* F1-score

For a deep overview of many different evaluation methods, see the Sklearn documentation: [https://scikit-learn.org/stable/modules/model_evaluation.html](https://scikit-learn.org/stable/modules/model_evaluation.html)

In [32]:
# Get baseline results with personal function
baseline_results = evaluate_classification_metrics(y_true=val_labels, y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

Video N°235: Model1: Building, fitting and evaluating our first deep model on text data

### Model 1: A simple dense model (feed-forward neural network)

In [33]:
# Create tensorboard callback (need to create a new one for each model)
# import personal 'create_tensorboard_callback' function

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [34]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [35]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers 
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation function
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [36]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [37]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240425-071837
Epoch 1/5


2024-04-25 07:18:38.780162: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f913c4f0450 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-25 07:18:38.780192: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050, Compute Capability 8.6
2024-04-25 07:18:38.802005: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-25 07:18:38.839442: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1714022318.907023    4804 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


215/215 [==============================] - 8s 32ms/step - loss: 0.6121 - accuracy: 0.6891 - val_loss: 0.5358 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.4421 - accuracy: 0.8156 - val_loss: 0.4701 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3469 - accuracy: 0.8609 - val_loss: 0.4568 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2836 - accuracy: 0.8902 - val_loss: 0.4620 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2377 - accuracy: 0.9121 - val_loss: 0.4840 - val_accuracy: 0.7848


In [38]:
# Evaluate model 1
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4840 - accuracy: 0.7848


[0.4839674234390259, 0.7847769260406494]

In [39]:
# Look at the first 10 predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.30211535],
       [0.81263053],
       [0.99793977],
       [0.14240497],
       [0.10003319],
       [0.9369116 ],
       [0.9163297 ],
       [0.99439126],
       [0.95919824],
       [0.30103454]], dtype=float32)

In [40]:
# Convert model prediction
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [41]:
# Caluclate our model_1 results
model_1_results = evaluate_classification_metrics(y_true=val_labels,
                                                  y_pred=model_1_preds)
model_1_results

{'accuracy': 78.4776902887139,
 'precision': 0.7904782347685264,
 'recall': 0.7847769028871391,
 'f1': 0.7814477888250547}

In [42]:
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Nous n'avons donc pas de meilleurs résultats avec model_1 comparé à model_0

Video N°236: Visualising our model's learned word embeddings with TensorFlow's projector tool

### Visualizing learned embeddings

To further understand word embeddings, let's visualize them, to do so, we'll get the weights matrix (embedding matrix) from our embedding layer and visualize it using the Embedding project tool, see the TensorFlow guide for more: https://www.tensorflow.org/tutorials/text/word_embeddings

In [43]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [44]:
len(words_in_vocab)

10000

In [45]:
max_vocab_length

10000

In [46]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [47]:
# Get the weight matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been learned for 5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)


Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handy tool called projector: [https://projector.tensorflow.org/](https://projector.tensorflow.org/)

And TensorFlow also has an incredible guide on word embeddings themselves: [https://www.tensorflow.org/text/guide/word_embeddings](https://www.tensorflow.org/text/guide/word_embeddings)

In [48]:
# Create embedding files (we got this from TensorFlow's word embedding documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

If you are running this tutorial in Colaboratory, you can use the following snippet to download these files to your local machine (or use the file browser, View -> Table of contents -> File browser).
```python
# Download files from Colab to upload projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass
```

Our visual word embeddings might not look like much to us, but they help our model understand the relationships between words.

For more on a popular type of word embedding and more visual explanations check out the illustrated word2vec: [https://jalammar.github.io/illustrated-word2vec/](https://jalammar.github.io/illustrated-word2vec/)

Video N°237: High-level overview of Recurrent Networks (RNNs) + where to learn more

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

**Resources:** If you want an overview of the internals of a recurrent neural network, see the following:
- MIT's sequence modelling lecture [https://www.youtube.com/watch?v=qjrad0V0uJE](https://www.youtube.com/watch?v=qjrad0V0uJE)
- Chris Olah's intro to LSTMs: [https://colah.github.io/posts/2015-08-Understanding-LSTMs/](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- Andrej Karpathy's the unreasonable effectiveness of recurrent neural networks: [https://karpathy.github.io/2015/05/21/rnn-effectiveness/](https://karpathy.github.io/2015/05/21/rnn-effectiveness/)

Video N°238: Model 2: Building, fitting and evaluating our first TensorFlow RNN model (LSTM)

### Model 2:LSTM

LSTM = long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:
```
Input (text) -> Tokenize -> Embedding -> Layers (RNN/dense) -> Output (label probability)
```

In [63]:
# Create an LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
print(f"input: A 3D tensor with shape [batch, timesteps, feature]")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
#x = layers.LSTM(units=64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to return_sequences=True
#print(x.shape)

x = layers.LSTM(units=64)(x)
print(f"x: A 2D tensor with shape [batch, features]")
print(f"After LSTM cell: {x.shape}")
#x = layers.Dense(units=64, activation="relu")(x)
#print(f"After Dense layer: {x.shape}")


outputs = layers.Dense(1, activation="sigmoid")(x)#binary data -> sigmoid
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

input: A 3D tensor with shape [batch, timesteps, feature]
After embedding: (None, 15, 128)
x: A 2D tensor with shape [batch, features]
After LSTM cell: (None, 64)


**Resource:** [https://ml-cheatsheet.readthedocs.io/en/latest/activation_functions.html#tanh](https://ml-cheatsheet.readthedocs.io/en/latest/activation_functions.html#tanh)

In [64]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [65]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [66]:
# Fit the model
history_2 = model_2.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="Model_2_LSTM")])

Saving TensorBoard log files to: model_logs/Model_2_LSTM/20240425-084100
Epoch 1/5
215/215 [==============================] - 9s 36ms/step - loss: 0.2228 - accuracy: 0.9223 - val_loss: 0.5823 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1541 - accuracy: 0.9439 - val_loss: 0.6515 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1278 - accuracy: 0.9511 - val_loss: 0.6535 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1033 - accuracy: 0.9618 - val_loss: 0.7478 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0864 - accuracy: 0.9673 - val_loss: 0.9948 - val_accuracy: 0.7769


In [67]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[2.2915702e-03],
       [9.1071284e-01],
       [9.9974161e-01],
       [3.1918209e-02],
       [3.2051542e-04],
       [9.9545503e-01],
       [7.8616792e-01],
       [9.9978596e-01],
       [9.9962962e-01],
       [4.7671857e-01]], dtype=float32)

In [69]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [70]:
# Calculate model 2 results
model_2_results = evaluate_classification_metrics(y_true=val_labels,
                                                  y_pred=model_2_preds)
model_2_results

{'accuracy': 77.69028871391076,
 'precision': 0.7798819103655997,
 'recall': 0.7769028871391076,
 'f1': 0.7743498017353971}

In [71]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [72]:
np.array(list(model_2_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Nous n'avons donc pas de meilleurs résultats avec model_2 comparé à model_0

Video N°239: Model 3: Building, fitting and evaluating our first TensorFlow RNN model(LSTM)

### Model 3: GRU

Another popular and effective RNN component is the GRU or 'gated recurrent unit'.

The GRU cell has similar features to an LSTM cell but has less parameters.


In [78]:
# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype=tf.string)
print(f"input: A 3D tensor with shape [batch, timesteps, feature]")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")

x = layers.GRU(units=64)(x)
print(f"After GRU 64 layer: {x.shape}")

#x = layers.GRU(units=64, return_sequences=True)(x) # if you want to stack recurrent layers on top of each other, you need return_sequences=True
#print(f"After GRU 64 layer: {x.shape}")

#x = layers.LSTM(units=64, return_sequences=True)(x)
#print(f"Ater LSTM 64 layer: {x.shape}")

# x = layers.GRU(units=64)(x)
# print(f"After GRU 64 layer: {x.shape}")

#x = layers.Dense(units=64, activation="relu")(x)
#print(f"After Dense 64 layer: {x.shape}")

outputs = layers.Dense(1, activation="sigmoid")(x)
print(f"After Dense 1 layer: {x.shape}")
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

input: A 3D tensor with shape [batch, timesteps, feature]
After embedding: (None, 15, 128)
After GRU 64 layer: (None, 64)
After Dense 1 layer: (None, 64)


In [79]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_5 (GRU)                 (None, 64)                37248     
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [80]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [81]:
# Fit the model
model_3_history = model_3.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240425-093013
Epoch 1/5
215/215 [==============================] - 8s 29ms/step - loss: 0.1602 - accuracy: 0.9378 - val_loss: 0.7030 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0806 - accuracy: 0.9698 - val_loss: 0.8951 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0691 - accuracy: 0.9742 - val_loss: 1.1263 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0571 - accuracy: 0.9756 - val_loss: 0.9133 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0525 - accuracy: 0.9774 - val_loss: 1.1386 - val_accuracy: 0.7664


In [82]:
# Make some predictions with our GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[5.8805733e-04],
       [8.5248351e-01],
       [9.9977404e-01],
       [7.9319023e-02],
       [7.4843869e-05],
       [9.9663627e-01],
       [4.0089709e-01],
       [9.9990082e-01],
       [9.9984097e-01],
       [5.2246863e-01]], dtype=float32)

In [85]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [86]:
# Calculate model 3 results
model_3_results = evaluate_classification_metrics(y_true=val_labels,
                                                  y_pred=model_3_preds)
model_3_results

{'accuracy': 76.64041994750657,
 'precision': 0.771218657508237,
 'recall': 0.7664041994750657,
 'f1': 0.7627908927491446}

In [87]:
np.array(list(model_3_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Nous n'avons donc pas de meilleurs résultats avec model_3 comparé à model_0

Video N°240: Model 4: Building, fitting and evaluating a bidirectional RNN model

### Model 4: Bidirectional RNN

Normal RNN's go from left to right (just like you'd read an English sentence) however, a bidirectional RNN goes from right to left as well as left to right.

In [92]:
# Building a bidirectional RNN in TensorFlow
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layer=layers.LSTM(units=64, return_sequences=True))(x)
x = layers.Bidirectional(layer=layers.LSTM(units=64))(x)
print(f"64 neurons give 128 parameters because they are bidirectional: {x.shape}")
outputs = layers.Dense(units=1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

64 neurons give 128 parameters because they are bidirectional: (None, 128)


In [93]:
# Get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_4 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable par

In [94]:
# Compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [95]:
# Fit the model
model_4_history = model_4.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240425-101251
Epoch 1/5
215/215 [==============================] - 10s 34ms/step - loss: 0.1026 - accuracy: 0.9723 - val_loss: 0.9223 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0498 - accuracy: 0.9781 - val_loss: 1.0296 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0456 - accuracy: 0.9796 - val_loss: 1.2900 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0453 - accuracy: 0.9803 - val_loss: 1.2059 - val_accuracy: 0.7520
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0375 - accuracy: 0.9815 - val_loss: 1.5789 - val_accuracy: 0.7585


In [96]:
# Make predictions with our bidirectional model
model_4_pred_probs = model_4.predict(x=val_sentences)
model_4_pred_probs

24/24 [==============================] - 1s 2ms/step


array([[4.82321009e-02],
       [7.32800841e-01],
       [9.99981523e-01],
       [2.51118660e-01],
       [9.70484780e-06],
       [9.99954343e-01],
       [9.95988786e-01],
       [9.99993563e-01],
       [9.99987602e-01],
       [9.97826993e-01],
       [4.07975131e-05],
       [9.98018742e-01],
       [2.53750804e-05],
       [1.52779967e-01],
       [1.24250564e-05],
       [1.16791553e-03],
       [1.87875994e-05],
       [2.57878019e-05],
       [3.77196223e-02],
       [9.99920249e-01],
       [9.99969840e-01],
       [1.55574380e-05],
       [9.99953628e-01],
       [1.22624639e-04],
       [9.99972224e-01],
       [9.99991179e-01],
       [5.00596325e-05],
       [2.00350754e-04],
       [3.82125232e-04],
       [9.34236050e-01],
       [9.99645829e-01],
       [2.90070398e-04],
       [9.89477057e-03],
       [4.76437068e-04],
       [8.86642516e-01],
       [5.61519563e-01],
       [9.99979854e-01],
       [3.21135193e-01],
       [1.68866292e-02],
       [9.99982357e-01],


In [97]:
# Convert pred probs to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [99]:
# Calculate the results of our bidirectional model
model_4_results = evaluate_classification_metrics(y_true=val_labels,
                                                  y_pred=model_4_preds)
model_4_results

{'accuracy': 75.8530183727034,
 'precision': 0.7581222241586,
 'recall': 0.7585301837270341,
 'f1': 0.7581368118458878}

In [101]:
np.array(list(model_4_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Nous n'avons donc pas de meilleurs résultats avec model_4 comparé à model_0

In [102]:
np.array(list(model_4_results.values())) > np.array(list(model_3_results.values()))

array([False, False, False, False])

Et même moins bien que l'unidirectionnel

Video N°241: Discussing the intuition behind Conv1D neural networks for text and sequences

## Convolutional Neural Networks for text (and other types of sequences)

We've used CNN's for images but images are typically 2D (height x width)... however, our text data is 1D.

Previously we've Conv2D for our image data but now we're going to use Conv1D.

The typical structure of a Conv1D model for sequences (in our case, text):

```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + pooling) -> Outputs (class probabilities)
```

### Model 5: Conv1D

For different explanations of parameters see:
* https://poloclub.github.io/cnn-explainer/ (this is for 2D but can relate to 1D data)
* Difference between "same" and "valid" padding: 
https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t

    "VALID" = without padding:

       inputs:         1  2  3  4  5  6  7  8  9  10 11 (12 13)
                      |________________|                dropped
                                     |_________________|

    "SAME" = with zero padding:

                   pad|                                      |pad
       inputs:      0 |1  2  3  4  5  6  7  8  9  10 11 12 13|0  0
                   |________________|
                                  |_________________|
                                                 |________________|

In this example:

    Input width = 13
    Filter width = 6
    Stride = 5

Notes:

    "VALID" only ever drops the right-most columns (or bottom-most rows).
    "SAME" tries to pad evenly left and right, but if the amount of columns to be added is odd, it will add the extra column to the right, as is the case in this example (the same logic applies vertically: there may be an extra row of zeros at the bottom).

Edit:

About the name:

    With "SAME" padding, if you use a stride of 1, the layer's outputs will have the same spatial dimensions as its inputs.
    With "VALID" padding, there's no "made-up" padding inputs. The layer only uses valid input data.


In [106]:
# Test out our embedding layer, Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sequence into embedding 
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, # this is also referred to as an ngram of 5 (meaning it looks at 5 words at a time)
                        strides=1, # default
                        activation="relu",
                        padding="same") # default = 'valid', the output is smaller than the input shape, 'same' means output is same shape as input
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to "get the most important feature" or "get the feature with the highest value"
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

In [107]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01333897, -0.05173763, -0.02926391, ..., -0.07142751,
          0.05672431,  0.01856982],
        [ 0.0237428 ,  0.02525576,  0.00799904, ..., -0.01024315,
         -0.00361403,  0.03839456],
        [-0.02869729, -0.03602771, -0.00440447, ...,  0.01082572,
          0.04177393,  0.05831759],
        ...,
        [ 0.00970045, -0.01046002, -0.0209655 , ..., -0.01571367,
          0.01658685,  0.01973417],
        [ 0.00970045, -0.01046002, -0.0209655 , ..., -0.01571367,
          0.01658685,  0.01973417],
        [ 0.00970045, -0.01046002, -0.0209655 , ..., -0.01571367,
          0.01658685,  0.01973417]]], dtype=float32)>

In [108]:
conv_1d_output

<tf.Tensor: shape=(1, 15, 32), dtype=float32, numpy=
array([[[0.00516754, 0.02805864, 0.        , 0.        , 0.05524044,
         0.05397297, 0.06454626, 0.        , 0.00899472, 0.        ,
         0.        , 0.00348191, 0.        , 0.        , 0.01064136,
         0.04497818, 0.        , 0.02408688, 0.09160361, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.03213548,
         0.        , 0.04826054],
        [0.02420677, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.00024258, 0.        , 0.        ,
         0.03855972, 0.        , 0.07779754, 0.02658998, 0.        ,
         0.        , 0.        , 0.06230744, 0.01617962, 0.03493495,
         0.00051936, 0.        , 0.        , 0.07279091, 0.        ,
         0.02811832, 0.        , 0.00202857, 0.        , 0.        ,
         0.        , 0.0025794 ],
        [0.        , 0.        , 0.        , 0.    

In [109]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.02420677, 0.09043157, 0.01359839, 0.06675692, 0.05524044,
        0.07433862, 0.06454626, 0.09060487, 0.03699689, 0.05421348,
        0.04297315, 0.06289778, 0.07779754, 0.04061557, 0.06478347,
        0.04497818, 0.        , 0.06902144, 0.1256347 , 0.06967805,
        0.04198116, 0.035947  , 0.        , 0.07279091, 0.12716058,
        0.04735307, 0.06786746, 0.02364375, 0.01521179, 0.0408621 ,
        0.04505679, 0.06849273]], dtype=float32)>

Video N°242: Model 5: Building, fitting and evaluating a 1D CNN for text

In [111]:
# Create 1-dimensional convolutional layer to model sequences
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64,
                  kernel_size=5,
                  strides=1,
                  activation="relu",
                  padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(units=64, activation='relu')(x)
outputs = layers.Dense(units=1, activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# Compile Conv1D
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

# Get a summary of our Conv1D model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_4 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_4 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_16 (Dense)            (None, 1)              

In [112]:
# Fit the model
model_5_history = model_5.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20240425-120643
Epoch 1/5
215/215 [==============================] - 10s 37ms/step - loss: 0.1294 - accuracy: 0.9609 - val_loss: 0.8772 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0744 - accuracy: 0.9729 - val_loss: 1.0631 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0605 - accuracy: 0.9777 - val_loss: 1.1207 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0551 - accuracy: 0.9796 - val_loss: 1.1976 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0514 - accuracy: 0.9796 - val_loss: 1.1838 - val_accuracy: 0.7559


In [113]:
# Make some predictions with our Conv1D model
model_5_pred_probs = model_5.predict(x=val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[5.3564584e-01],
       [9.4461179e-01],
       [9.9992359e-01],
       [7.3626623e-02],
       [1.7251563e-06],
       [9.9461752e-01],
       [9.8255402e-01],
       [9.9997795e-01],
       [9.9999964e-01],
       [9.3640131e-01]], dtype=float32)

In [114]:
# Convert model 5 pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [116]:
# Calculate results for model 5
model_5_results = evaluate_classification_metrics(y_true=val_labels,
                                                  y_pred=model_5_preds)
model_5_results

{'accuracy': 75.59055118110236,
 'precision': 0.7555220084761928,
 'recall': 0.7559055118110236,
 'f1': 0.7555828084930034}

In [117]:
np.array(list(model_5_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Nous n'avons donc pas de meilleurs résultats avec model_5 comparé à model_0

Video N°243: Using TensorFlow Hub for pretrained word embeddings (transfer learning for NLP)

## Model 6: TensorFlow Hub Pretrained Sentence Encoder

Now we've built a few of our own model's, let's try and use transfer learning for NLP, specifically using TensorFlow Hub's Universal Sentence Encoder:
- [https://tfhub.dev/google/universal-sentence-encoder/4](https://tfhub.dev/google/universal-sentence-encoder/4)
- [https://www.kaggle.com/models/google/universal-sentence-encoder](https://www.kaggle.com/models/google/universal-sentence-encoder)

See how the USE was created here: [https://arxiv.org/abs/1803.11175](https://arxiv.org/abs/1803.11175)

In [118]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence, "When you can the universal sentence encoder on a sentence, it turns it into numbers."])
print(embed_samples[0][:50])

tf.Tensor(
[-0.01157027  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680985  0.05589838 -0.01068729 -0.00597292  0.00639323 -0.0181952
  0.00030814  0.09105888  0.05874645 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865346 -0.04209305  0.0267898   0.03011008  0.00321069
 -0.00337971 -0.04787356  0.02266719 -0.00985925 -0.04063613 -0.01292093
 -0.04666384  0.056303   -0.03949255  0.00517688  0.02495828 -0.07014441
  0.02871508  0.04947684 -0.00633978 -0.08960193  0.02807117 -0.00808362
 -0.01360601  0.0599865  -0.10361787 -0.05195374  0.00232955 -0.0233253
 -0.03758105  0.03327729], shape=(50,), dtype=float32)


In [119]:
embed_samples[0].shape

TensorShape([512])

Video N°244: Model 6: Building, training and evaluating a transfer learning model for NLP

In [ ]:
# Create a Keras Layer using the USE pretrained layer from tensorflow hub 